In [ ]:
!nvidia-smi

Mon Jul 31 18:04:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchtext==0.14.1
!pip install torchdata==0.5.1
!python -m pip install "optimum[openvino,nncf]"
!pip install diffusers
!pip install accelerate
!pip install tomesd
!pip install nncf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 943.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu1

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%%capture
!sudo apt -qq install git-lfs
!git config --global credential.helper store

In [ ]:
!git config --global user.email "nnkhoinguyen9@gmail.com"
!git config --global user.name "Zero-nnkn"

# OpenVINO
- Graph optimization
- Post-training static quantization
- Quantization Aware Training (QAT)
- FP16 (half precision)
- Pruning
- Knowledge Distillation

## ToMe + QAE + KD

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/train-optimize-sd-intel/overview.png)

### Script

In [ ]:
# %%writefile train_text_to_image_qat.py
# import argparse
# import itertools
# import logging
# import math
# import os
# import random
# import tempfile
# from copy import deepcopy
# from functools import partial
# from io import BytesIO
# from pathlib import Path
# from typing import Any, Dict, Iterable, List, Optional, Tuple, Union

# import numpy as np
# import requests
# import tomesd
# import torch
# import torch.nn.functional as F
# import torch.utils.checkpoint
# from accelerate import Accelerator
# from accelerate.logging import get_logger
# from accelerate.utils import set_seed
# from datasets import load_dataset
# from diffusers import DDIMScheduler, DDPMScheduler, DiffusionPipeline, LMSDiscreteScheduler, StableDiffusionPipeline
# from diffusers.optimization import get_scheduler
# from huggingface_hub import HfFolder, Repository, whoami
# from nncf import NNCFConfig
# from nncf.common.logging import nncf_logger
# from nncf.torch import create_compressed_model, register_default_init_args
# from nncf.torch.initialization import PTInitializingDataLoader
# from nncf.torch.layer_utils import CompressionParameter
# from openvino._offline_transformations import apply_moc_transformations, compress_quantize_weights_transformation
# from PIL import Image
# from requests.packages.urllib3.exceptions import InsecureRequestWarning
# from torchvision import transforms
# from tqdm import tqdm

# from optimum.exporters.onnx import export, get_stable_diffusion_models_for_export
# from optimum.intel import OVStableDiffusionPipeline
# from optimum.utils import (
#     DIFFUSION_MODEL_TEXT_ENCODER_SUBFOLDER,
#     DIFFUSION_MODEL_UNET_SUBFOLDER,
#     DIFFUSION_MODEL_VAE_DECODER_SUBFOLDER,
#     DIFFUSION_MODEL_VAE_ENCODER_SUBFOLDER,
# )


# def export_models(
#     models_and_onnx_configs: Dict[
#         str, Tuple[Union["PreTrainedModel", "TFPreTrainedModel", "ModelMixin"], "OnnxConfig"]
#     ],
#     output_dir: Path,
#     opset: Optional[int] = None,
#     output_names: Optional[List[str]] = None,
#     device: str = "cpu",
#     input_shapes: Optional[Dict] = None,
#     disable_dynamic_axes_fix: Optional[bool] = False,
#     dtype: Optional[str] = None,
#     model_kwargs: Optional[Dict[str, Any]] = None,
# ) -> Tuple[List[List[str]], List[List[str]]]:
#     """
#     Exports a Pytorch or TensorFlow encoder decoder model to an ONNX Intermediate Representation.
#     The following method exports the encoder and decoder components of the model as separate
#     ONNX files.

#     Args:
#         models_and_onnx_configs (`Dict[str, Tuple[Union[`PreTrainedModel`, `TFPreTrainedModel`, `ModelMixin`], `OnnxConfig`]]):
#             A dictionnary containing the models to export and their corresponding onnx configs.
#         output_dir (`Path`):
#             Output directory to store the exported ONNX models.
#         opset (`Optional[int]`, defaults to `None`):
#             The version of the ONNX operator set to use.
#         output_names (`Optional[List[str]]`, defaults to `None`):
#             The names to use for the exported ONNX files. The order must be the same as the order of submodels in the ordered dict `models_and_onnx_configs`.
#             If None, will use the keys from `models_and_onnx_configs` as names.
#         device (`str`, defaults to `"cpu"`):
#             The device on which the ONNX model will be exported. Either `cpu` or `cuda`. Only PyTorch is supported for
#             export on CUDA devices.
#         input_shapes (`Optional[Dict]`, defaults to `None`):
#             If specified, allows to use specific shapes for the example input provided to the ONNX exporter.
#         disable_dynamic_axes_fix (`Optional[bool]`, defaults to `False`):
#             Whether to disable the default dynamic axes fixing.
#         dtype (`Optional[str]`, defaults to `None`):
#             Data type to remap the model inputs to. PyTorch-only. Only `fp16` is supported.
#         model_kwargs (`Optional[Dict[str, Any]]`, defaults to `None`):
#             Experimental usage: keyword arguments to pass to the model during
#             the export. This argument should be used along the `custom_onnx_config` argument
#             in case, for example, the model inputs/outputs are changed (for example, if
#             `model_kwargs={"output_attentions": True}` is passed).
#     Returns:
#         `Tuple[List[List[str]], List[List[str]]]`: A tuple with an ordered list of the model's inputs, and the named
#         outputs from the ONNX configuration.
#     """
#     outputs = []

#     if output_names is not None and len(output_names) != len(models_and_onnx_configs):
#         raise ValueError(
#             f"Provided custom names {output_names} for the export of {len(models_and_onnx_configs)} models. Please provide the same number of names as models to export."
#         )

#     for i, model_name in enumerate(models_and_onnx_configs.keys()):
#         print(model_name)
#         submodel, sub_onnx_config = models_and_onnx_configs[model_name]
#         output_name = output_names[i] if output_names is not None else Path(model_name + ".onnx")

#         output_path = output_dir / output_name
#         output_path.parent.mkdir(parents=True, exist_ok=True)

#         outputs.append(
#             export(
#                 model=submodel,
#                 config=sub_onnx_config,
#                 output=output_path,
#                 opset=opset,
#                 device=device,
#                 input_shapes=input_shapes,
#                 disable_dynamic_axes_fix=disable_dynamic_axes_fix,
#                 dtype=dtype,
#                 model_kwargs=model_kwargs,
#             )
#         )

#     outputs = list(map(list, zip(*outputs)))
#     return outputs

# requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# random.seed(42)
# logger = get_logger(__name__)
# nncf_logger.setLevel(logging.INFO)


# def get_full_repo_name(model_id: str, organization: Optional[str] = None, token: Optional[str] = None):
#     if token is None:
#         token = HfFolder.get_token()
#     if organization is None:
#         username = whoami(token)["name"]
#         return f"{username}/{model_id}"
#     else:
#         return f"{organization}/{model_id}"


# def pokemon_preprocess_train(examples, train_transforms, tokenize_captions, image_column="image"):
#     image = examples[image_column]
#     examples["pixel_values"] = train_transforms(image.convert("RGB"))
#     examples["input_ids"] = tokenize_captions(examples)
#     return examples


# def get_pil_from_url(url):
#     response = requests.get(url, verify=False, timeout=20)
#     image = Image.open(BytesIO(response.content))
#     return image.convert("RGB")


# # Many of the images in laion2B dataset are unavailable
# # This is a workaround to substitute such images with a backup or cached available examples
# BACKUP_PAIR = (
#     get_pil_from_url(
#         "https://thumbs.dreamstime.com/t/altai-mountains-mountain-lake-russia-siberia-chuya-ridge-49130812.jpg"
#     ),
#     "Altai mountains Stock Photography",
# )
# AVAILABLE_EXAMPLES = []


# def check_text_data(data):
#     if isinstance(data, str):
#         return True
#     if isinstance(data, list):
#         return all(isinstance(x, str) for x in data)
#     return False


# def laion2B_preprocess_train(examples, train_transforms, tokenize_captions, image_column="URL", text_column="TEXT"):
#     url = examples[image_column]
#     try:
#         image = get_pil_from_url(url)
#         if not check_text_data(examples[text_column]):
#             raise ValueError("Text data is not valid")
#         AVAILABLE_EXAMPLES.append((url, examples[text_column]))
#     except Exception:
#         logger.info(f"Can't load image from url: {url}, using cache with size: {len(AVAILABLE_EXAMPLES)}")
#         if len(AVAILABLE_EXAMPLES) > 0:
#             backup_id = random.randint(0, len(AVAILABLE_EXAMPLES) - 1)
#             backup_example = AVAILABLE_EXAMPLES[backup_id]
#             try:
#                 image = get_pil_from_url(backup_example[0])
#                 examples[text_column] = backup_example[1]
#             except Exception:
#                 logger.info(f"Can't load image from cached url: {backup_example[0]}, using backup")
#                 image = BACKUP_PAIR[0].copy()
#                 examples[text_column] = BACKUP_PAIR[1]
#         else:
#             logger.info(f"Can't load image from url: {url}, using backup")
#             image = BACKUP_PAIR[0].copy()
#             examples[text_column] = BACKUP_PAIR[1]

#     examples["pixel_values"] = train_transforms(image)
#     examples["input_ids"] = tokenize_captions(examples)
#     return examples


# dataset_name_mapping = {
#     "lambdalabs/pokemon-blip-captions": {
#         "columns": ("image", "text"),
#         "preprocess_fn": pokemon_preprocess_train,
#         "streaming": False,
#     },
#     "laion/laion2B-en": {
#         "columns": ("URL", "TEXT"),
#         "preprocess_fn": laion2B_preprocess_train,
#         "streaming": True,
#     },
#     "laion/laion2B-en-aesthetic": {
#         "columns": ("URL", "TEXT"),
#         "preprocess_fn": laion2B_preprocess_train,
#         "streaming": True,
#     },
#     "laion/laion-art": {
#         "columns": ("URL", "TEXT"),
#         "preprocess_fn": laion2B_preprocess_train,
#         "streaming": True,
#     },
#     "laion/laion400m": {
#         "columns": ("url", "caption"),
#         "preprocess_fn": partial(laion2B_preprocess_train, image_column="url", text_column="caption"),
#         "streaming": True,
#     },
# }


# # Adapted from torch-ema https://github.com/fadel/pytorch_ema/blob/master/torch_ema/ema.py#L14
# class EMAQUnet:
#     """
#     Exponential Moving Average of unets weights
#     """

#     def __init__(self, parameters: Iterable[torch.nn.Parameter], decay=0.9999):
#         parameters = list(parameters)
#         self.shadow_params = [p.clone().detach() for p in parameters]

#         self.decay = decay
#         self.optimization_step = 0

#     def get_decay(self, optimization_step):
#         """
#         Compute the decay factor for the exponential moving average.
#         """
#         value = (1 + optimization_step) / (10 + optimization_step)
#         return 1 - min(self.decay, value)

#     @torch.no_grad()
#     def step(self, parameters):
#         parameters = list(parameters)

#         self.optimization_step += 1
#         self.decay = self.get_decay(self.optimization_step)

#         for s_param, param in zip(self.shadow_params, parameters):
#             if param.requires_grad:
#                 tmp = param.clone()
#                 tmp = tmp.to(s_param.device)
#                 # tmp = self.decay * (s_param - param.clone.to(s_param.device))
#                 tmp.sub_(s_param)
#                 tmp.mul_(self.decay)
#                 tmp.neg_()
#                 s_param.sub_(tmp)
#             else:
#                 s_param.copy_(param)

#         torch.cuda.empty_cache()

#     def copy_to(self, parameters: Iterable[torch.nn.Parameter]) -> None:
#         """
#         Copy current averaged parameters into given collection of parameters.

#         Args:
#             parameters: Iterable of `torch.nn.Parameter`; the parameters to be
#                 updated with the stored moving averages. If `None`, the
#                 parameters with which this `ExponentialMovingAverage` was
#                 initialized will be used.
#         """
#         parameters = list(parameters)
#         for s_param, param in zip(self.shadow_params, parameters):
#             param.data.copy_(s_param.data)

#     def to(self, device=None, dtype=None) -> None:
#         r"""Move internal buffers of the ExponentialMovingAverage to `device`.

#         Args:
#             device: like `device` argument to `torch.Tensor.to`
#         """
#         # .to() on the tensors handles None correctly
#         self.shadow_params = [
#             p.to(device=device, dtype=dtype) if p.is_floating_point() else p.to(device=device)
#             for p in self.shadow_params
#         ]


# def parse_args():
#     parser = argparse.ArgumentParser(description="Stable Diffusion 8-bit Quantization for OpenVINO")
#     parser.add_argument(
#         "--model_id",
#         type=str,
#         default=None,
#         required=True,
#         help="Path to pretrained model or model identifier from huggingface.co/models.",
#     )
#     parser.add_argument(
#         "--revision",
#         type=str,
#         default=None,
#         required=False,
#         help="Revision of pretrained model identifier from huggingface.co/models.",
#     )
#     parser.add_argument(
#         "--dataset_name",
#         type=str,
#         default="lambdalabs/pokemon-blip-captions",
#         help=(
#             "The name of the Dataset (from the HuggingFace hub) to train on (could be your own, possibly private,"
#             " dataset). It can also be a path pointing to a local copy of a dataset in your filesystem,"
#             " or to a folder containing files that 🤗 Datasets can understand."
#         ),
#     )
#     parser.add_argument(
#         "--dataset_config_name",
#         type=str,
#         default=None,
#         help="The config of the Dataset, leave as None if there's only one config.",
#     )
#     parser.add_argument(
#         "--train_data_dir",
#         type=str,
#         default=None,
#         help=(
#             "A folder containing the training data. Folder contents must follow the structure described in"
#             " https://huggingface.co/docs/datasets/image_dataset#imagefolder. In particular, a `metadata.jsonl` file"
#             " must exist to provide the captions for the images. Ignored if `dataset_name` is specified."
#         ),
#     )
#     parser.add_argument(
#         "--max_train_samples",
#         type=int,
#         default=None,
#         help=(
#             "For debugging purposes or quicker training, truncate the number of training examples to this "
#             "value if set."
#         ),
#     )
#     parser.add_argument(
#         "--output_dir",
#         type=str,
#         default="sd-model-quantized",
#         help="The output directory where the model predictions and checkpoints will be written.",
#     )
#     parser.add_argument(
#         "--cache_dir",
#         type=str,
#         default=None,
#         help="The directory where the downloaded models and datasets will be stored.",
#     )
#     parser.add_argument("--seed", type=int, default=42, help="A seed for reproducible training.")
#     parser.add_argument(
#         "--resolution",
#         type=int,
#         default=512,
#         help=(
#             "The resolution for input images, all the images in the train/validation dataset will be resized to this"
#             " resolution"
#         ),
#     )
#     parser.add_argument(
#         "--noise_scheduler",
#         type=str,
#         default=None,
#         choices=["DDIM", "DDPM", "LMSDiscrete"],
#         help="The noise scheduler for the Diffusion pipiline used for training.",
#     )
#     parser.add_argument(
#         "--beta_start",
#         type=float,
#         default=0.00085,
#         help="Beta min value for noise scheduler.",
#     )
#     parser.add_argument(
#         "--beta_end",
#         type=float,
#         default=0.012,
#         help="BetaMax value for noise scheduler.",
#     )
#     parser.add_argument(
#         "--beta_schedule",
#         type=str,
#         default="scaled_linear",
#         help="Beta schedule type",
#     )
#     parser.add_argument(
#         "--noise_schedule_steps",
#         type=int,
#         default=1000,
#         help=("The noise scheduler max train timestemps"),
#     )
#     parser.add_argument(
#         "--center_crop",
#         default=False,
#         action="store_true",
#         help=(
#             "Whether to center crop the input images to the resolution. If not set, the images will be randomly"
#             " cropped. The images will be resized to the resolution first before cropping."
#         ),
#     )
#     parser.add_argument(
#         "--random_flip",
#         action="store_true",
#         help="whether to randomly flip images horizontally",
#     )
#     parser.add_argument(
#         "--train_batch_size", type=int, default=1, help="Batch size (per device) for the training dataloader."
#     )
#     parser.add_argument("--num_train_epochs", type=int, default=1)
#     parser.add_argument(
#         "--max_train_steps",
#         type=int,
#         default=15000,
#         help="Total number of training steps to perform.  If provided, overrides num_train_epochs.",
#     )
#     parser.add_argument(
#         "--gradient_accumulation_steps",
#         type=int,
#         default=4,
#         help="Number of updates steps to accumulate before performing a backward/update pass.",
#     )
#     parser.add_argument(
#         "--gradient_checkpointing",
#         action="store_true",
#         help="Whether or not to use gradient checkpointing to save memory at the expense of slower backward pass.",
#     )
#     parser.add_argument(
#         "--learning_rate",
#         type=float,
#         default=1e-5,
#         help="Initial learning rate (after the potential warmup period) to use.",
#     )
#     parser.add_argument(
#         "--scale_lr",
#         action="store_true",
#         help="Scale the learning rate by the number of GPUs, gradient accumulation steps, and batch size.",
#     )
#     parser.add_argument(
#         "--lr_scheduler",
#         type=str,
#         default="constant",
#         help=(
#             'The scheduler type to use. Choose between ["linear", "cosine", "cosine_with_restarts", "polynomial",'
#             ' "constant", "constant_with_warmup"]'
#         ),
#     )
#     parser.add_argument(
#         "--lr_warmup_steps", type=int, default=0, help="Number of steps for the warmup in the lr scheduler."
#     )
#     parser.add_argument(
#         "--snr_gamma",
#         type=float,
#         default=None,
#         help="SNR weighting gamma to be used if rebalancing the loss. Recommended value is 5.0. "
#         "More details here: https://arxiv.org/abs/2303.09556.",
#     )
#     parser.add_argument(
#         "--use_8bit_adam", action="store_true", help="Whether or not to use 8-bit Adam from bitsandbytes."
#     )
#     parser.add_argument(
#         "--allow_tf32",
#         action="store_true",
#         help=(
#             "Whether or not to allow TF32 on Ampere GPUs. Can be used to speed up training. For more information, see"
#             " https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices"
#         ),
#     )
#     parser.add_argument(
#         "--ema_device",
#         type=str,
#         default=None,
#         choices=["cpu", "cuda"],
#         help="Whether to use EMA model and where to store the EMA model.",
#     )
#     parser.add_argument(
#         "--quantization_mode",
#         type=str,
#         default="moderate",
#         choices=["moderate", "aggressive"],
#         help=(
#             "'aggressive' mode quantizes all MatMul operations while 'moderate' keeps MatMul that applies attention mask non-quantized."
#             " The later allows preserving a better accuracy while keeping the similar inference performance after optimization."
#         ),
#     )
#     parser.add_argument(
#         "--non_ema_revision",
#         type=str,
#         default=None,
#         required=False,
#         help=(
#             "Revision of pretrained non-ema model identifier. Must be a branch, tag or git identifier of the local or"
#             " remote repository specified with --pretrained_model_name_or_path."
#         ),
#     )
#     parser.add_argument(
#         "--dataloader_num_workers",
#         type=int,
#         default=0,
#         help=(
#             "Number of subprocesses to use for data loading. 0 means that the data will be loaded in the main process."
#         ),
#     )
#     parser.add_argument("--adam_beta1", type=float, default=0.9, help="The beta1 parameter for the Adam optimizer.")
#     parser.add_argument("--adam_beta2", type=float, default=0.999, help="The beta2 parameter for the Adam optimizer.")
#     parser.add_argument("--adam_weight_decay", type=float, default=1e-2, help="Weight decay to use.")
#     parser.add_argument("--adam_epsilon", type=float, default=1e-08, help="Epsilon value for the Adam optimizer")
#     parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
#     parser.add_argument("--push_to_hub", action="store_true", help="Whether or not to push the model to the Hub.")
#     parser.add_argument("--hub_token", type=str, default=None, help="The token to use to push to the Model Hub.")
#     parser.add_argument(
#         "--hub_model_id",
#         type=str,
#         default=None,
#         help="The name of the repository to keep in sync with the local `output_dir`.",
#     )
#     parser.add_argument(
#         "--logging_dir",
#         type=str,
#         default="logs",
#         help=(
#             "[TensorBoard](https://www.tensorflow.org/tensorboard) log directory. Will default to"
#             " *output_dir/runs/**CURRENT_DATETIME_HOSTNAME***."
#         ),
#     )
#     parser.add_argument(
#         "--mixed_precision",
#         type=str,
#         default=None,
#         choices=["no", "fp16", "bf16"],
#         help=(
#             "Whether to use mixed precision. Choose between fp16 and bf16 (bfloat16). Bf16 requires PyTorch >="
#             " 1.10.and an Nvidia Ampere GPU.  Default to the value of accelerate config of the current system or the"
#             " flag passed with the `accelerate.launch` command. Use this argument to override the accelerate config."
#         ),
#     )
#     parser.add_argument(
#         "--report_to",
#         type=str,
#         default="tensorboard",
#         help=(
#             'The integration to report the results and logs to. Supported platforms are `"tensorboard"`'
#             ' (default), `"wandb"` and `"comet_ml"`. Use `"all"` to report to all integrations.'
#         ),
#     )
#     parser.add_argument("--local_rank", type=int, default=-1, help="For distributed training: local_rank")
#     parser.add_argument(
#         "--checkpointing_steps",
#         type=int,
#         default=15000,
#         help=(
#             "Save a checkpoint of the training state every X updates. These checkpoints are only suitable for resuming"
#             " training using `--resume_from_checkpoint`."
#         ),
#     )
#     parser.add_argument(
#         "--checkpoints_total_limit",
#         type=int,
#         default=1,
#         help=(
#             "Max number of checkpoints to store. Passed as `total_limit` to the `Accelerator` `ProjectConfiguration`."
#             " See Accelerator::save_state https://huggingface.co/docs/accelerate/package_reference/accelerator#accelerate.Accelerator.save_state"
#             " for more docs"
#         ),
#     )
#     parser.add_argument(
#         "--resume_from_checkpoint",
#         type=str,
#         default=None,
#         help=(
#             "Whether training should be resumed from a previous checkpoint. Use a path saved by"
#             ' `--checkpointing_steps`, or `"latest"` to automatically select the last available checkpoint.'
#         ),
#     )
#     parser.add_argument(
#         "--tome_ratio",
#         type=float,
#         default=0,
#         help=(
#             "Token Merging ratio. If 0, no merging is applied" "More details here: https://arxiv.org/abs/2303.17604."
#         ),
#     )
#     parser.add_argument(
#         "--opt_init_steps",
#         type=int,
#         default=300,
#         help=("Max number of initialization steps for quantization before the actual fine-tuning."),
#     )
#     parser.add_argument(
#         "--opt_init_type",
#         type=str,
#         default="mean_min_max",
#         choices=["min_max", "mean_min_max", "threesigma"],
#         help="They way how to estimate activation quantization paramters at the initializatin step before QAT.",
#     )
#     parser.add_argument(
#         "--tune_quantizers_only",
#         action="store_true",
#         default=False,
#         help="Whether to train quantization parameters only.",
#     )
#     parser.add_argument("--use_kd", action="store_true", help="Use Knowledge Distillation to boost accuracy.")

#     args = parser.parse_args()
#     env_local_rank = int(os.environ.get("LOCAL_RANK", -1))
#     if env_local_rank != -1 and env_local_rank != args.local_rank:
#         args.local_rank = env_local_rank

#     # Sanity checks
#     if args.dataset_name is None and args.train_data_dir is None:
#         raise ValueError("Need either a dataset name or a training folder.")

#     # default to using the same revision for the non-ema model if not specified
#     if args.non_ema_revision is None:
#         args.non_ema_revision = args.revision
#     return args


# def get_noise_scheduler(args):
#     scheduler_args = {
#         "beta_start": args.beta_start,
#         "beta_end": args.beta_end,
#         "beta_schedule": args.beta_schedule,
#         "num_train_timesteps": args.noise_schedule_steps,
#     }
#     if args.noise_scheduler == "DDIM":
#         noise_scheduler = DDIMScheduler(**scheduler_args)
#     elif args.noise_scheduler == "DDPM":
#         noise_scheduler = DDPMScheduler(**scheduler_args)
#     elif args.noise_scheduler == "LMSDiscrete":
#         noise_scheduler = LMSDiscreteScheduler(**scheduler_args)
#     else:
#         raise ValueError(f"Unknown noise schedule {args.noise_schedule}")
#     return noise_scheduler


# def export_to_onnx(pipeline, save_dir, device='cpu', opset=14):
#     unet = pipeline.unet
#     vae = pipeline.vae
#     text_encoder = pipeline.text_encoder
#     unet.eval().to(device)
#     vae.eval().to(device)
#     text_encoder.eval().to(device)

#     ONNX_WEIGHTS_NAME = "model.onnx"

#     output_names = [
#         os.path.join(DIFFUSION_MODEL_TEXT_ENCODER_SUBFOLDER, ONNX_WEIGHTS_NAME),
#         os.path.join(DIFFUSION_MODEL_UNET_SUBFOLDER, ONNX_WEIGHTS_NAME),
#         os.path.join(DIFFUSION_MODEL_VAE_ENCODER_SUBFOLDER, ONNX_WEIGHTS_NAME),
#         os.path.join(DIFFUSION_MODEL_VAE_DECODER_SUBFOLDER, ONNX_WEIGHTS_NAME),
#     ]

#     with torch.no_grad():
#         models_and_onnx_configs = get_stable_diffusion_models_for_export(pipeline)
#         pipeline.save_config(save_dir)
#         export_models(
#             models_and_onnx_configs=models_and_onnx_configs, output_dir=Path(save_dir), output_names=output_names, device=device, opset=14
#         )


# def export_to_openvino(pipeline, onnx_dir, save_dir):
#     ov_pipe = OVStableDiffusionPipeline.from_pretrained(
#         model_id=onnx_dir,
#         from_onnx=True,
#         model_save_dir=save_dir,
#         tokenizer=pipeline.tokenizer,
#         scheduler=pipeline.scheduler,
#         feature_extractor=pipeline.feature_extractor,
#         compile=False,
#     )
#     apply_moc_transformations(ov_pipe.unet.model, cf=False)
#     compress_quantize_weights_transformation(ov_pipe.unet.model)
#     ov_pipe.save_pretrained(save_dir)


# class UnetInitDataset(torch.utils.data.Dataset):
#     def __init__(self, data):
#         super().__init__()
#         self.init_data = data

#     def __len__(self):
#         return len(self.init_data)

#     def __getitem__(self, index):
#         return self.init_data[index]


# def prepare_nncf_init_data(pipeline, dataloader, args):
#     weight_dtype = torch.float32
#     text_encoder = pipeline.text_encoder
#     vae = pipeline.vae
#     noise_scheduler = pipeline.scheduler

#     nncf_init_data = []

#     logger.info(f"Fetching {args.opt_init_steps} for the initialization...")
#     for _, batch in tqdm(zip(range(args.opt_init_steps), itertools.islice(dataloader, 0, args.opt_init_steps))):
#         with torch.no_grad():
#             # Convert images to latent space
#             latents = vae.encode(batch["pixel_values"].to(weight_dtype)).latent_dist.sample()
#             latents = latents * 0.18215

#             # Sample noise that we'll add to the latents
#             noise = torch.randn_like(latents)
#             bsz = latents.shape[0]
#             # Sample a random timestep for each image
#             timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bsz,), device=latents.device)
#             timesteps = timesteps.long()

#             # Add noise to the latents according to the noise magnitude at each timestep
#             # (this is the forward diffusion process)
#             noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
#             encoder_hidden_states = text_encoder(batch["input_ids"])[0]
#             nncf_init_data.append(
#                 (
#                     torch.squeeze(noisy_latents).to("cpu"),
#                     torch.squeeze(timesteps).to("cpu"),
#                     torch.squeeze(encoder_hidden_states).to("cpu"),
#                     0,
#                 )
#             )
#     return nncf_init_data


# # The config should work for Stable Diffusion v1.4-2.1
# def get_nncf_config(pipeline, dataloader, args):
#     text_encoder = pipeline.text_encoder
#     unet = pipeline.unet
#     moderate_quantization_config = {
#         "input_info": [
#             {  # "keyword": "latent_model_input",
#                 "sample_size": [1, unet.config["in_channels"], unet.config["sample_size"], unet.config["sample_size"]]
#             },
#             {"sample_size": [1]},  # "keyword": "t",
#             {  # "keyword": "encoder_hidden_states",
#                 "sample_size": [1, text_encoder.config.max_position_embeddings, text_encoder.config.hidden_size]
#             },
#         ],
#         "log_dir": args.output_dir,  # The log directory for NNCF-specific logging outputs.
#         "compression": [
#             {
#                 "algorithm": "quantization",  # Specify the algorithm here.
#                 "preset": "mixed",
#                 "initializer": {
#                     "range": {"num_init_samples": args.opt_init_steps, "type": args.opt_init_type},
#                     "batchnorm_adaptation": {"num_bn_adaptation_samples": args.opt_init_steps},
#                 },
#                 "scope_overrides": {"activations": {"{re}.*baddbmm_0": {"mode": "symmetric"}}},
#                 "ignored_scopes": [
#                     "{re}.*__add___[0-2]",
#                     "{re}.*layer_norm_0",
#                     "{re}.*Attention.*/bmm_0",
#                     "{re}.*__truediv__*",
#                     "{re}.*group_norm_0",
#                     "{re}.*mul___[0-2]",
#                     "{re}.*silu_[0-2]",
#                 ],
#                 "overflow_fix": "disable",
#                 "export_to_onnx_standard_ops": True,
#             },
#         ],
#     }

#     aggressive_quantization_config = {
#         "input_info": [
#             {  # "keyword": "latent_model_input",
#                 "sample_size": [1, unet.config["in_channels"], unet.config["sample_size"], unet.config["sample_size"]]
#             },
#             {"sample_size": [1]},  # "keyword": "t",
#             {  # "keyword": "encoder_hidden_states",
#                 "sample_size": [1, text_encoder.config.max_position_embeddings, text_encoder.config.hidden_size]
#             },
#         ],
#         "log_dir": args.output_dir,  # The log directory for NNCF-specific logging outputs.
#         "compression": [
#             {
#                 "algorithm": "quantization",  # Specify the algorithm here.
#                 "preset": "mixed",
#                 "initializer": {
#                     "range": {"num_init_samples": args.opt_init_steps, "type": args.opt_init_type},
#                     "batchnorm_adaptation": {"num_bn_adaptation_samples": args.opt_init_steps},
#                 },
#                 "scope_overrides": {
#                     "activations": {"{re}.*baddbmm_0": {"mode": "symmetric"}, "{re}.*bmm_0": {"mode": "symmetric"}}
#                 },
#                 "ignored_scopes": [
#                     "{re}.*layer_norm_0",
#                     "{re}.*__truediv__*",
#                     "{re}.*group_norm_0",
#                     "{re}.*mul___[0-2]",
#                     "{re}.*silu_[0-2]",
#                 ],
#                 "overflow_fix": "disable",
#                 "export_to_onnx_standard_ops": True,
#             },
#         ],
#     }

#     class UnetInitDataLoader(PTInitializingDataLoader):
#         def get_inputs(self, dataloader_output):
#             noisy_latents = dataloader_output[0].float().to(unet.device, non_blocking=True)
#             timesteps = dataloader_output[1].float().to(unet.device, non_blocking=True)
#             encoder_hidden_states = dataloader_output[2].float().to(unet.device, non_blocking=True)
#             return (noisy_latents, timesteps, encoder_hidden_states), {}

#         def get_target(self, dataloader_output):
#             return dataloader_output[0]

#     quantization_config = (
#         aggressive_quantization_config if args.quantization_mode == "aggressive" else moderate_quantization_config
#     )

#     nncf_config = NNCFConfig.from_dict(quantization_config)
#     nncf_config = register_default_init_args(nncf_config, UnetInitDataLoader(dataloader))
#     # nncf_config = register_default_init_args(nncf_config, dataloader)
#     return nncf_config


# def collate_fn(examples, preprocess_fn, tokenizer):
#     examples = [preprocess_fn(example) for example in examples]
#     pixel_values = torch.stack([example["pixel_values"] for example in examples])
#     pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
#     input_ids = [example["input_ids"] for example in examples]
#     padded_tokens = tokenizer.pad({"input_ids": input_ids}, padding=True, return_tensors="pt")
#     return {
#         "pixel_values": pixel_values,
#         "input_ids": padded_tokens.input_ids,
#         "attention_mask": padded_tokens.attention_mask,
#     }

# # We need to tokenize input captions and transform the images.
# def tokenize_captions(examples, tokenizer, caption_column, is_train=True):
#     captions = []
#     caption = examples[caption_column]
#     if isinstance(caption, str):
#         captions.append(caption)
#     elif isinstance(caption, (list, np.ndarray)):
#         # take a random caption if there are multiple
#         captions.append(random.choice(caption) if is_train else caption[0])
#     else:
#         raise ValueError(f"Caption column `{caption_column}` should contain either strings or lists of strings.")
#     inputs = tokenizer(captions[0], max_length=tokenizer.model_max_length, padding="do_not_pad", truncation=True)
#     input_ids = inputs.input_ids
#     return input_ids

# def main():
#     args = parse_args()
#     logging_dir = os.path.join(args.output_dir, args.logging_dir)

#     weight_dtype = torch.float32
#     if args.mixed_precision == "fp16":
#         weight_dtype = torch.float16
#     elif args.mixed_precision == "bf16":
#         weight_dtype = torch.bfloat16

#     accelerator = Accelerator(
#         gradient_accumulation_steps=args.gradient_accumulation_steps,
#         mixed_precision=args.mixed_precision,
#         log_with=args.report_to,
#         project_dir=logging_dir,
#     )

#     logging.basicConfig(
#         format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
#         datefmt="%m/%d/%Y %H:%M:%S",
#         level=logging.INFO,
#     )

#     logger.info(accelerator.state, main_process_only=False)

#     # If passed along, set the training seed now.
#     if args.seed is not None:
#         set_seed(args.seed)

#     # Handle the repository creation
#     if accelerator.is_main_process:
#         if args.push_to_hub:
#             if args.hub_model_id is None:
#                 repo_name = get_full_repo_name(Path(args.output_dir).name, token=args.hub_token)
#             else:
#                 repo_name = args.hub_model_id
#             repo = Repository(args.output_dir, clone_from=repo_name)

#             with open(os.path.join(args.output_dir, ".gitignore"), "w+") as gitignore:
#                 if "step_*" not in gitignore:
#                     gitignore.write("step_*\n")
#                 if "epoch_*" not in gitignore:
#                     gitignore.write("epoch_*\n")
#         elif args.output_dir is not None:
#             os.makedirs(args.output_dir, exist_ok=True)

#     pipeline = DiffusionPipeline.from_pretrained(args.model_id)

#     if args.use_kd:
#         teacher_model = deepcopy(pipeline.unet)

#     if args.tome_ratio > 0:
#         logger.info(f"Using Token Merging with ratio: {args.tome_ratio}")
#         tomesd.apply_patch(
#             pipeline, ratio=args.tome_ratio, use_rand=False
#         )  # Can also use pipe.unet in place of pipe here

#     # Load models and create wrapper for stable diffusion
#     tokenizer = pipeline.tokenizer
#     text_encoder = pipeline.text_encoder
#     vae = pipeline.vae
#     unet = pipeline.unet
#     noise_scheduler = pipeline.scheduler
#     if args.noise_scheduler:
#         noise_scheduler = get_noise_scheduler(args)

#     # Freeze vae and text_encoder
#     vae.requires_grad_(False)
#     text_encoder.requires_grad_(False)

#     if args.gradient_checkpointing:
#         unet.enable_gradient_checkpointing()

#     if args.scale_lr:
#         args.learning_rate = (
#             args.learning_rate * args.gradient_accumulation_steps * args.train_batch_size * accelerator.num_processes
#         )

#     # Initialize the optimizer
#     if args.use_8bit_adam:
#         try:
#             import bitsandbytes as bnb
#         except ImportError:
#             raise ImportError(
#                 "Please install bitsandbytes to use 8-bit Adam. You can do so by running `pip install bitsandbytes`"
#             )

#         optimizer_cls = bnb.optim.AdamW8bit
#     else:
#         optimizer_cls = torch.optim.AdamW

#     # Get the datasets: you can either provide your own training and evaluation files (see below)
#     # or specify a Dataset from the hub (the dataset will be downloaded automatically from the datasets Hub).

#     # In distributed training, the load_dataset function guarantees that only one local process can concurrently
#     # download the dataset.
#     dataset_settings = dataset_name_mapping.get(args.dataset_name, None)
#     if dataset_settings is None:
#         raise ValueError(
#             f"Dataset {args.dataset_name} not supported. Please choose from {dataset_name_mapping.keys()}"
#         )

#     if args.dataset_name is not None:
#         # Downloading and loading a dataset from the hub.
#         dataset = load_dataset(
#             args.dataset_name,
#             args.dataset_config_name,
#             cache_dir=args.cache_dir,
#             streaming=dataset_settings["streaming"],
#         )
#     else:
#         data_files = {}
#         if args.train_data_dir is not None:
#             data_files["train"] = os.path.join(args.train_data_dir, "**")
#         dataset = load_dataset(
#             "imagefolder",
#             data_files=data_files,
#             cache_dir=args.cache_dir,
#         )
#         # See more about loading custom images at
#         # https://huggingface.co/docs/datasets/v2.4.0/en/image_load#imagefolder

#     # Preprocessing the datasets.
#     # We need to tokenize inputs and targets.

#     # 6. Get the column names for input/target.
#     dataset_columns = dataset_settings["columns"]
#     caption_column = dataset_columns[1]

#     # Preprocessing the datasets.
#     train_transforms = transforms.Compose(
#         [
#             transforms.Resize((args.resolution, args.resolution), interpolation=transforms.InterpolationMode.BILINEAR),
#             transforms.CenterCrop(args.resolution) if args.center_crop else transforms.RandomCrop(args.resolution),
#             transforms.RandomHorizontalFlip() if args.random_flip else transforms.Lambda(lambda x: x),
#             transforms.ToTensor(),
#             transforms.Normalize([0.5], [0.5]),
#         ]
#     )

#     preprocess_fn = partial(
#         dataset_settings["preprocess_fn"], train_transforms=train_transforms, tokenize_captions=partial(tokenize_captions, tokenizer=tokenizer, caption_column=caption_column)
#     )

#     with accelerator.main_process_first():
#         if args.max_train_samples is not None:
#             dataset["train"] = dataset["train"].shuffle(seed=42, buffer_size=args.max_train_samples)
#         # Set the training transforms
#         train_dataset = dataset["train"]

#     train_dataloader = torch.utils.data.DataLoader(
#         train_dataset, collate_fn=partial(collate_fn, preprocess_fn=preprocess_fn, tokenizer=tokenizer), batch_size=args.train_batch_size, num_workers=args.dataloader_num_workers
#     )

#     unet = accelerator.prepare(unet)
#     vae.to(unet.device)
#     text_encoder.to(unet.device)
#     train_dataloader = accelerator.prepare_data_loader(train_dataloader)
#     orig_unet = unet  # save link to original unet model for EMA

#     ## Create initialization dataset for PTQ
#     nncf_init_data = prepare_nncf_init_data(pipeline, train_dataloader, args)
#     init_dataloader = torch.utils.data.DataLoader(UnetInitDataset(nncf_init_data), batch_size=1, num_workers=1)
#     nncf_config = get_nncf_config(pipeline, init_dataloader, args)

#     # Quantize the model and initialize quantizer using init data
#     compression_controller, unet = create_compressed_model(unet, nncf_config)

#     statistics_unet = compression_controller.statistics()
#     logger.info(statistics_unet.to_str())

#     del nncf_init_data, init_dataloader
#     torch.cuda.empty_cache()

#     unet.train()

#     if args.tune_quantizers_only:
#         for p in unet.parameters():
#             if not isinstance(p, CompressionParameter):
#                 p.requires_grad = False

#     # Reinit
#     optimizer = optimizer_cls(
#         filter(lambda p: p.requires_grad, unet.parameters()),
#         lr=args.learning_rate,
#         betas=(args.adam_beta1, args.adam_beta2),
#         weight_decay=args.adam_weight_decay,
#         eps=args.adam_epsilon,
#     )
#     lr_scheduler = get_scheduler(
#         args.lr_scheduler,
#         optimizer=optimizer,
#         num_warmup_steps=args.lr_warmup_steps * args.gradient_accumulation_steps,
#         num_training_steps=args.max_train_steps * args.gradient_accumulation_steps,
#     )

#     # Scheduler and math around the number of training steps.
#     overrode_max_train_steps = False
#     dataset_len = args.max_train_samples if args.max_train_samples is not None else len(train_dataloader)
#     num_update_steps_per_epoch = math.ceil(dataset_len / args.gradient_accumulation_steps)
#     if args.max_train_steps is None:
#         args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
#         overrode_max_train_steps = True

#     unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
#         unet, optimizer, train_dataloader, lr_scheduler
#     )

#     # Move text_encode and vae to gpu.
#     # For mixed precision training we cast the text_encoder and vae weights to half-precision
#     # as these models are only used for inference, keeping weights in full precision is not required.
#     text_encoder.to(accelerator.device, dtype=weight_dtype)
#     vae.to(accelerator.device, dtype=weight_dtype)
#     if args.use_kd:
#       teacher_model.to(accelerator.device, dtype=weight_dtype)

#     # Create EMA for the unet.
#     if args.ema_device:
#         ema_unet = EMAQUnet(orig_unet.parameters())
#         if args.ema_device == "cpu":
#             ema_unet.to("cpu")

#     # We need to recalculate our total training steps as the size of the training dataloader may have changed.
#     num_update_steps_per_epoch = math.ceil(dataset_len / args.gradient_accumulation_steps)
#     if overrode_max_train_steps:
#         args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
#     # Afterwards we recalculate our number of training epochs
#     args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

#     # We need to initialize the trackers we use, and also store our configuration.
#     # The trackers initializes automatically on the main process.
#     if accelerator.is_main_process:
#         accelerator.init_trackers("text2image-fine-tune", config=vars(args))

#     # Train!
#     total_batch_size = args.train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

#     logger.info("***** Running training *****")
#     logger.info(f"  Num examples = {dataset_len}")
#     logger.info(f"  Num Epochs = {args.num_train_epochs}")
#     logger.info(f"  Instantaneous batch size per device = {args.train_batch_size}")
#     logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
#     logger.info(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
#     logger.info(f"  Total optimization steps = {args.max_train_steps}")

#     # Only show the progress bar once on each machine.
#     progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
#     progress_bar.set_description("Steps")
#     global_step = 0

#     for epoch in range(args.num_train_epochs):
#         train_loss = 0.0
#         compression_controller.scheduler.epoch_step()

#         for step, batch in enumerate(train_dataloader):
#             with accelerator.accumulate(unet):
#                 # Convert images to latent space
#                 latents = vae.encode(batch["pixel_values"].to(weight_dtype)).latent_dist.sample()
#                 latents = latents * 0.18215

#                 # Sample noise that we'll add to the latents
#                 noise = torch.randn_like(latents)
#                 bsz = latents.shape[0]
#                 # Sample a random timestep for each image
#                 timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bsz,), device=latents.device)
#                 timesteps = timesteps.long()

#                 # Add noise to the latents according to the noise magnitude at each timestep
#                 # (this is the forward diffusion process)
#                 noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

#                 # Get the text embedding for conditioning
#                 encoder_hidden_states = text_encoder(batch["input_ids"])[0]

#                 # Predict the noise residual and compute loss
#                 noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

#                 loss = F.mse_loss(noise_pred.float(), noise.float(), reduction="mean")

#                 if args.use_kd:
#                     with torch.no_grad():
#                         orig_output = teacher_model(noisy_latents, timesteps, encoder_hidden_states).sample
#                     loss += F.mse_loss(noise_pred.float(), orig_output.float(), reduction="mean")

#                 # Gather the losses across all processes for logging (if we use distributed training).
#                 avg_loss = accelerator.gather(loss.repeat(args.train_batch_size)).mean()
#                 train_loss += avg_loss.item() / args.gradient_accumulation_steps

#                 compression_loss_unet = compression_controller.loss()
#                 loss = loss + compression_loss_unet

#                 # Backpropagate
#                 accelerator.backward(loss)
#                 if accelerator.sync_gradients:
#                     accelerator.clip_grad_norm_(unet.parameters(), args.max_grad_norm)
#                 optimizer.step()
#                 lr_scheduler.step()
#                 optimizer.zero_grad()

#             # Checks if the accelerator has performed an optimization step behind the scenes
#             if accelerator.sync_gradients:
#                 if args.ema_device:
#                     ema_unet.step(orig_unet.parameters())
#                 progress_bar.update(1)
#                 global_step += 1
#                 accelerator.log({"train_loss": train_loss}, step=global_step)
#                 train_loss = 0.0

#             logs = {"step_loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
#             progress_bar.set_postfix(**logs)

#             if global_step >= args.max_train_steps:
#                 break

#     if args.use_kd:
#         del teacher_model
#     # Create the pipeline using the trained modules and save it.
#     accelerator.wait_for_everyone()
#     if accelerator.is_main_process:
#         unet = accelerator.unwrap_model(unet)
#         if args.ema_device:
#             ema_unet.copy_to(orig_unet.parameters())

#     accelerator.end_training()

#     # Export optimized pipline to OpenVINO
#     export_unet = compression_controller.strip(do_copy=False)
#     export_pipeline = StableDiffusionPipeline(
#         text_encoder=text_encoder,
#         vae=vae,
#         unet=export_unet,
#         tokenizer=tokenizer,
#         scheduler=noise_scheduler,
#         safety_checker=pipeline.safety_checker,
#         feature_extractor=pipeline.feature_extractor,
#     )

#     print('Save model')
#     export_pipeline.save_pretrained(Path(args.output_dir))

#     if args.push_to_hub:
#         repo.push_to_hub(commit_message="End of training", blocking=False, auto_lfs_prune=True)
#     # print('Export to ONNX')
#     # export_to_onnx(export_pipeline, Path(args.output_dir) / "onnx", device='cuda', opset=14)
#     # print('Export to OpenVINO')
#     # export_to_openvino(export_pipeline, Path(args.output_dir) / "onnx", Path(args.output_dir) / "openvino")


# if __name__ == "__main__":
#     main()

### Run
- svjack/Stable-Diffusion-Pokemon-en
- justinpinkney/pokemon-stable-diffusion
- --gradient_checkpointing \ not support
- --use_kd \
    


In [ ]:
# !python train_text_to_image_qat.py \
#     --ema_device="cpu" \
#     --model_id="svjack/Stable-Diffusion-Pokemon-en" \
#     --center_crop \
#     --random_flip \
#     --train_batch_size=1 \
#     --dataloader_num_workers=1 \
#     --dataset_name="lambdalabs/pokemon-blip-captions" \
#     --max_train_steps=0 \
#     --opt_init_steps=300 \
#     --tome_ratio=0.5 \
#     --quantization_mode="moderate" \
#     --mixed_precision="no" \
#     --output_dir=sd-pokemon-static-quantized-tome \
#     --hub_model_id Zero-nnkn/sd-pokemon-static-quantized-tome \
#     --push_to_hub

## SQ + ToMe

In [ ]:
from pathlib import Path
from functools import partial
from tqdm import tqdm

import tomesd
from diffusers import StableDiffusionPipeline
from optimum.intel import OVConfig, OVQuantizer

SEED = 42
TOME_RATIO = 0.5

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


In [ ]:
model_id = "svjack/Stable-Diffusion-Pokemon-en"
pipeline = StableDiffusionPipeline.from_pretrained(model_id)

safety_checker/model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:128: FutureWarning: The configuration file of this scheduler: LMSDiscreteScheduler {
  "_class_name": "LMSDiscreteScheduler",
  "_diffusers_version": "0.19.3",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "steps_offset": 0,
  "timestep_spacing": "linspace",
  "trained_betas": null,
  "use_karras_sigmas": false
}
 is outdated. `steps_offset` should be set to 1 instead of 

In [ ]:
tomesd.apply_patch(
    pipeline, ratio=TOME_RATIO, use_rand=False,
)  # Can also use pipe.unet in place of pipe here

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.19.3",
  "_name_or_path": "svjack/Stable-Diffusion-Pokemon-en",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "LMSDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
# Quantizer
tokenizer = pipeline.tokenizer
unet = pipeline.unet
# quantization_config = OVConfig()
# quantizer = OVQuantizer.from_pretrained(pipeline.unet)

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torchvision import transforms
from transformers import DataCollator

NUM_SAMPLES = 100
BATCH_SIZE = 1
INIT_TYPE = 'min_max' # ["min_max", "mean_min_max", "threesigma"]

In [ ]:
# Dataset
from datasets import load_dataset


def tokenize_captions(examples, tokenizer, is_train=True):
    captions = []
    caption = examples[caption_column]
    if isinstance(caption, str):
        captions.append(caption)
    elif isinstance(caption, (list, np.ndarray)):
        # take a random caption if there are multiple
        captions.append(random.choice(caption) if is_train else caption[0])
    else:
        raise ValueError(f"Caption column `{caption_column}` should contain either strings or lists of strings.")
    inputs = tokenizer(captions[0], max_length=tokenizer.model_max_length, padding="do_not_pad", truncation=True)
    input_ids = inputs.input_ids
    return input_ids

def preprocess_dataset(examples, tokenizer, transforms, image_column="image"):
    image = examples[image_column]
    examples["pixel_values"] = transforms(image.convert("RGB"))
    examples["input_ids"] = tokenize_captions(examples, tokenizer=tokenizer)
    return examples

def collate_fn(examples, preprocess_fn, tokenizer):
    examples = [preprocess_fn(example) for example in examples]
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    pixel_values = pixel_values.to(memory_format=torch.contiguous_format).float()
    input_ids = [example["input_ids"] for example in examples]
    padded_tokens = tokenizer.pad({"input_ids": input_ids}, padding=True, return_tensors="pt")
    return {
        "pixel_values": pixel_values,
        "input_ids": padded_tokens.input_ids,
        "attention_mask": padded_tokens.attention_mask,
    }

train_transforms = transforms.Compose(
    [
        transforms.Resize((512, 512), interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(512),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

dataset_settings = {
  'columns': ('image', 'text'),
}
dataset_columns = dataset_settings['columns']
caption_column = dataset_columns[1]
preprocess_fn = partial(preprocess_dataset, tokenizer=tokenizer, transforms=train_transforms)

# Get dataset
dataset = load_dataset(
        'lambdalabs/pokemon-blip-captions',
        split='train',
        cache_dir=None,
    )
dataset = dataset.shuffle(seed=SEED).select(range(NUM_SAMPLES))

# Get dataloader
generator = torch.Generator()
generator.manual_seed(SEED)
sampler = RandomSampler(dataset, generator=generator)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=sampler, collate_fn=partial(collate_fn, preprocess_fn=preprocess_fn, tokenizer=tokenizer))

### Unet

In [ ]:
from nncf import NNCFConfig
from nncf.torch import create_compressed_model, register_default_init_args
from nncf.torch.dynamic_graph.io_handling import wrap_nncf_model_inputs_with_objwalk
from nncf.torch.initialization import PTInitializingDataLoader

DEVICE = 'cuda'

In [ ]:
class UnetDataset(Dataset):
    def __init__(self, data_list):
        super().__init__()
        self.data = data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]


class UnetInitDataLoader(PTInitializingDataLoader):
    def __init__(self, data_loader, device: str):
        super().__init__(data_loader)
        self.device = device

    def get_inputs(self, dataloader_output):
        noisy_latents = dataloader_output[1].float().to(self.device, non_blocking=True)
        timesteps = dataloader_output[2].float().to(self.device, non_blocking=True)
        encoder_hidden_states = dataloader_output[3].float().to(self.device, non_blocking=True)
        return (noisy_latents, timesteps, encoder_hidden_states), {}

    def get_target(self, dataloader_output):
        return dataloader_output[0]


def prepare_unet_data(pipeline, dataloader, device='cpu'):
    weight_dtype = torch.float32
    text_encoder = pipeline.text_encoder.to(device)
    vae = pipeline.vae.to(device)
    noise_scheduler = pipeline.scheduler

    unet_data = []

    for batch in tqdm(dataloader):
        with torch.no_grad():
            # Convert images to latent space
            latents = vae.encode(batch["pixel_values"].to(weight_dtype).to(device)).latent_dist.sample()
            latents = latents * 0.18215

            # Sample noise that we'll add to the latents
            noise = torch.randn_like(latents)
            bsz = latents.shape[0]
            # Sample a random timestep for each image
            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bsz,), device=latents.device)
            timesteps = timesteps.long()

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
            encoder_hidden_states = text_encoder(batch["input_ids"].to(device))[0]

            for i in range(len(noisy_latents)):
                unet_data.append(
                    (
                        noise[i].to('cpu'),
                        noisy_latents[i].to('cpu'),
                        timesteps[i].to('cpu'),
                        encoder_hidden_states[i].to('cpu'),
                    )
                )
    return unet_data

unet_dataset = UnetDataset(prepare_unet_data(pipeline, dataloader, device=DEVICE))
unet_dataloader = DataLoader(unet_dataset, batch_size=BATCH_SIZE)
unet_init_dataloader = UnetInitDataLoader(unet_dataloader, device=DEVICE)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:134: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'LMSDiscreteScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'LMSDiscreteScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
100%|██████████| 100/100 [00:32<00:00,  3.08it/s]


In [ ]:
DEFAULT_COMPRESSION = {
    "algorithm": "quantization",
    "preset": "mixed",
    "initializer": {
        "range": {"num_init_samples": NUM_SAMPLES, "type": INIT_TYPE},
        "batchnorm_adaptation": {"num_bn_adaptation_samples": 0},
    },
    "scope_overrides": {"activations": {"{re}.*matmul_0": {"mode": "symmetric"}}},
    "ignored_scopes": [
        "{re}.*Embedding.*",
        "{re}.*add___.*",
        "{re}.*layer_norm_.*",
        "{re}.*matmul_1",
        "{re}.*__truediv__.*",
    ],
    "overflow_fix": "disable",
    "export_to_onnx_standard_ops": True,
}

INT8_WEIGHT_COMPRESSION = {
    "algorithm": "quantization",
    "weights": {
        "mode": "symmetric",
        "bits": 8,
        "target_scopes": [
            "{re}.*Embedding.*",
            "{re}.*matmul_.*",
            "{re}.*addmm_.*",
            "{re}.*baddmm_.*",
            "{re}.*linear_.*",
        ],
        "ignored_scopes": [
            "{re}.*conv_*",
        ],
    },
    "activations": {
        "ignored_scopes": [
            "{re}.*add___.*",
            "{re}.*__radd___.*",
            "{re}.*layer_norm_.*",
            "{re}.*__truediv__.*",
            "{re}.*__mul___.*",
            "{re}.*__rmul___.*",
            "{re}.*tanh_.*",
            "{re}.*pow_.*",
            "{re}.*matmul_.*",
            "{re}.*addmm_.*",
            "{re}.*baddmm_.*",
            "{re}.*linear_.*",
            "{re}.*conv_.*",
        ],
    },
    "overflow_fix": "disable",
    "export_to_onnx_standard_ops": True,
}

MODERATE_COMPRESSION =  {
    "algorithm": "quantization",
    "preset": "mixed",
    "initializer": {
        "range": {"num_init_samples": NUM_SAMPLES, "type": INIT_TYPE},
        "batchnorm_adaptation": {"num_bn_adaptation_samples":  NUM_SAMPLES},
    },
    "scope_overrides": {"activations": {"{re}.*baddbmm_0": {"mode": "symmetric"}}},
    "ignored_scopes": [
        "{re}.*__add___[0-2]",
        "{re}.*layer_norm_0",
        "{re}.*Attention.*/bmm_0",
        "{re}.*__truediv__*",
        "{re}.*group_norm_0",
        "{re}.*mul___[0-2]",
        "{re}.*silu_[0-2]",
    ],
    "overflow_fix": "disable",
    "export_to_onnx_standard_ops": True,
}

AGGRESSIVE_COMPRESSION = {
    "algorithm": "quantization",  # Specify the algorithm here.
    "preset": "mixed",
    "initializer": {
        "range": {"num_init_samples": NUM_SAMPLES, "type": INIT_TYPE},
        "batchnorm_adaptation": {"num_bn_adaptation_samples":  NUM_SAMPLES},
    },
    "scope_overrides": {
        "activations": {"{re}.*baddbmm_0": {"mode": "symmetric"}, "{re}.*bmm_0": {"mode": "symmetric"}}
    },
    "ignored_scopes": [
        "{re}.*layer_norm_0",
        "{re}.*__truediv__*",
        "{re}.*group_norm_0",
        "{re}.*mul___[0-2]",
        "{re}.*silu_[0-2]",
    ],
    "overflow_fix": "disable",
    "export_to_onnx_standard_ops": True,
}


def prepare_unet_quantize_config(pipeline, compressions):
    config = {
        "input_info": [
            {  # "keyword": "latent_model_input",
                "sample_size": [1, pipeline.unet.config["in_channels"], pipeline.unet.config["sample_size"], pipeline.unet.config["sample_size"]]
            },
            {"sample_size": [1]},  # "keyword": "t",
            {  # "keyword": "encoder_hidden_states",
                "sample_size": [1, pipeline.text_encoder.config.max_position_embeddings, pipeline.text_encoder.config.hidden_size]
            },
        ],
        "compression": [c for c in compressions],
    }

    return config

quantization_config = prepare_unet_quantize_config(pipeline, compressions=[MODERATE_COMPRESSION])

In [ ]:
def quantize(model, quantization_config, init_dataloader):
    nncf_config = NNCFConfig.from_dict(quantization_config)
    nncf_config = register_default_init_args(nncf_config, init_dataloader)
    controller, compressed_model = create_compressed_model(
        model, nncf_config, wrap_inputs_fn=wrap_nncf_model_inputs_with_objwalk
    )
    compressed_model = controller.strip(do_copy=False)
    return compressed_model

In [ ]:
unet.to(DEVICE)

UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0): Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): LoRACompatibleConv(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): ToMeBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_features

In [ ]:
compressed_unet = quantize(unet, quantization_config=quantization_config, init_dataloader=unet_init_dataloader)

/usr/local/lib/python3.10/dist-packages/nncf/torch/dynamic_graph/wrappers.py:175: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1615.)
  result = operator(*args, **kwargs)


INFO:nncf:Not adding activation input quantizer for operation: 20 UNet2DConditionModel/ModuleList[down_blocks]/CrossAttnDownBlock2D[0]/ModuleList[resnets]/ResnetBlock2D[0]/NNCFGroupNorm[norm1]/group_norm_0
21 UNet2DConditionModel/ModuleList[down_blocks]/CrossAttnDownBlock2D[0]/ModuleList[resnets]/ResnetBlock2D[0]/SiLU[nonlinearity]/silu_0

INFO:nncf:Not adding activation input quantizer for operation: 8 UNet2DConditionModel/Timesteps[time_proj]/__mul___0
INFO:nncf:Not adding activation input quantizer for operation: 9 UNet2DConditionModel/Timesteps[time_proj]/__rmul___0
INFO:nncf:Not adding activation input quantizer for operation: 1043 UNet2DConditionModel/ModuleList[up_blocks]/CrossAttnUpBlock2D[2]/ModuleList[resnets]/ResnetBlock2D[1]/SiLU[nonlinearity]/silu_1
INFO:nncf:Not adding activation input quantizer for operation: 1123 UNet2DConditionModel/ModuleList[up_blocks]/CrossAttnUpBlock2D[2]/ModuleList[resnets]/ResnetBlock2D[2]/SiLU[nonlinearity]/silu_1
INFO:nncf:Not adding activation

/usr/local/lib/python3.10/dist-packages/nncf/torch/nncf_network.py:938: FutureWarning: Old style of accessing NNCF-specific attributes and methods on NNCFNetwork objects is deprecated. Access the NNCF-specific attrs through the NNCFInterface, which is set up as an `nncf` attribute on the compressed model object.
For instance, instead of `compressed_model.get_graph()` you should now write `compressed_model.nncf.get_graph()`.
The old style will be removed after NNCF v2.5.0
  warning_deprecated(


#### Export

In [ ]:
from huggingface_hub import Repository

exported_pipeline = StableDiffusionPipeline(
    text_encoder=pipeline.text_encoder,
    vae=pipeline.vae,
    unet=compressed_unet,
    tokenizer=pipeline.tokenizer,
    scheduler=pipeline.scheduler,
    safety_checker=pipeline.safety_checker,
    feature_extractor=pipeline.feature_extractor,
)

##### SDPipeline

In [ ]:
exported_pipeline.save_pretrained(save_dir)

/content/sd-pokemon-quantized-tome  is already a clone of https://huggingface.co/Zero-nnkn/sd-pokemon-quantized-tome. Make sure you pull the latest changes with `repo.git_pull()`.


##### ONNX

In [ ]:
import os
from optimum.exporters.onnx import export_models, get_stable_diffusion_models_for_export
from optimum.intel import OVStableDiffusionPipeline
from optimum.utils import (
    DIFFUSION_MODEL_TEXT_ENCODER_SUBFOLDER,
    DIFFUSION_MODEL_UNET_SUBFOLDER,
    DIFFUSION_MODEL_VAE_DECODER_SUBFOLDER,
    DIFFUSION_MODEL_VAE_ENCODER_SUBFOLDER,
)

def export_to_onnx(pipeline, save_dir):
    save_dir = Path(save_dir)
    save_dir.mkdir(parents=True, exist_ok=True)
    unet = pipeline.unet
    vae = pipeline.vae
    text_encoder = pipeline.text_encoder

    unet.eval()
    vae.eval()
    text_encoder.eval()

    ONNX_WEIGHTS_NAME = "model.onnx"

    output_names = [
        os.path.join(DIFFUSION_MODEL_TEXT_ENCODER_SUBFOLDER, ONNX_WEIGHTS_NAME),
        os.path.join(DIFFUSION_MODEL_UNET_SUBFOLDER, ONNX_WEIGHTS_NAME),
        os.path.join(DIFFUSION_MODEL_VAE_ENCODER_SUBFOLDER, ONNX_WEIGHTS_NAME),
        os.path.join(DIFFUSION_MODEL_VAE_DECODER_SUBFOLDER, ONNX_WEIGHTS_NAME),
    ]

    with torch.no_grad():
        models_and_onnx_configs = get_stable_diffusion_models_for_export(pipeline)
        pipeline.save_config(save_dir)
        export_models(
            models_and_onnx_configs=models_and_onnx_configs, output_dir=Path(save_dir), output_names=output_names, device='cuda'
        )

/content/sd-pokemon-static-quantized-tome-onnx is already a clone of https://huggingface.co/Zero-nnkn/sd-pokemon-static-quantized-tome-onnx. Make sure you pull the latest changes with `repo.git_pull()`.
Using framework PyTorch: 1.13.1+cu117
/usr/local/lib/python3.10/dist-packages/transformers/models/clip/modeling_clip.py:286: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.10/dist-packages/transformers/models/clip/modeling_clip.py:294: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to

NNCF relies on custom-wrapping the `forward` call in order to function properly.
Arbitrary adjustments to the forward function on an NNCFNetwork object have undefined behaviour.
If you need to replace the underlying forward function of the original model so that NNCF should be using that instead of the original forward function that NNCF saved during the compressed model creation, you can do this by calling:
model.nncf.set_original_unbound_forward(fn)
if `fn` has an unbound 0-th `self` argument, or
with model.nncf.temporary_bound_original_forward(fn): ...
if `fn` already had 0-th `self` argument bound or never had it in the first place.


/usr/local/lib/python3.10/dist-packages/diffusers/models/unet_2d_condition.py:752: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if any(s % default_overall_up_factor != 0 for s in sample.shape[-2:]):
/usr/local/lib/python3.10/dist-packages/tomesd/patch.py:13: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  downsample = int(math.ceil(math.sqrt(original_tokens // x.shape[1])))
/usr/local/lib/python3.10/dist-packages/tomesd/patch.py:18: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of

NNCF relies on custom-wrapping the `forward` call in order to function properly.
Arbitrary adjustments to the forward function on an NNCFNetwork object have undefined behaviour.
If you need to replace the underlying forward function of the original model so that NNCF should be using that instead of the original forward function that NNCF saved during the compressed model creation, you can do this by calling:
model.nncf.set_original_unbound_forward(fn)
if `fn` has an unbound 0-th `self` argument, or
with model.nncf.temporary_bound_original_forward(fn): ...
if `fn` already had 0-th `self` argument bound or never had it in the first place.


UnsupportedOperatorError: ignored